In [ ]:
import geoviews as gv
import cartopy.crs as ccrs

from geoviews import annotate

gv.extension('bokeh')

This notebook documents the usage and design of the `annotate` coordinator, which make it easy to draw, edit, and annotate polygon, polyline, rectangle, and point data on top of a map. The `annotate` function builds on Bokeh Drawing Tools connected to HoloViews drawing-tools streams, providing convenient access to the drawn data from Python. For a basic introduction to the ``annotate`` function see the [HoloViews Annotator user guide](https://holoviews.org/user_guide/Annotators.html). Importing GeoViews extends these annotators in a few important ways:

* The editable table will display coordinates as latitude and longitude pairs
* Additional checkpoint, restore and clear data tools are added
* The `gv.Path` annotator makes the distinction between feature nodes (at the start and end of a path) and regular nodes

In this guide we will demonstrate the usage of these annotators on a tile source to demonstrate how projections between Mercator and lat/lon coordinates are handled.

## Annotating Points

When annotating a GeoViews elements it is assumed that the data is displayed in Web Mercator coordinates (as is the default when working with the Bokeh backend) but the coordinates may be supplied in any coordinate system. The table used to edit the coordinates will however always display longitudes and latitudes making it simpler to edit the coordinate values.

In [ ]:
tiles = gv.tile_sources.Wikipedia()

sample_points = dict(
    Longitude = [-10131185, -10131943, -10131766, -10131032],
    Latitude  = [  3805587,   3803182,   3801073,   3799778])

points = gv.Points(sample_points, crs=ccrs.GOOGLE_MERCATOR).opts(
    size=10, line_color='black', padding=0.1, responsive=True, min_height=600)

point_annotate = annotate.instance()

annotated = point_annotate(points, annotations=['Size'])

annotate.compose(tiles, annotated)

The annotator will return coordinates in the coordinate system they were originally defined in:

In [ ]:
point_annotate.annotated.dframe()

## Annotating Rectangles

The GeoViews Rectangles annotator behaves much like the annotator in HoloViews but also projects the coordinates in the table to be more readable:

In [ ]:
rectangles = gv.Rectangles([(0, 0, 10, 10)])

box_annotate = annotate.instance()

annotated = box_annotate(rectangles)

annotate.compose(tiles, annotated)

In [ ]:
box_annotate.annotated.dframe()

## Annotating Paths

The path annotator behaves slightly differently to the one in HoloViews, when annotating a GeoViews Path a distinction is made between features nodes which describe the start and end point of a multi-line geometry and regular nodes which make up the interior nodes of the geometry. Additionally when attaching a new path on a regular node this will automatically split the existing path promoting the regular node to a feature node.

In [ ]:
sample_poly=dict(
    Longitude = [-10114986, -10123906, -10130333, -10121522, -10129889, -10122959],
    Latitude  = [  3806790,   3812413,   3807530,   3805407,   3798394,   3796693])


path = gv.Path([sample_poly], crs=ccrs.GOOGLE_MERCATOR).opts(
    padding=0.1, line_width=2, color='black', responsive=True)

path_annotate = annotate.instance()

annotated = path_annotate(path, vertex_annotations=['Height'])

annotate.compose(tiles, annotated)

Just like the HoloViews version each path can be accessed using `iloc` or by using split which will return a list of Path elements representing each geometry:

In [ ]:
path_annotate.annotated.iloc[0].dframe()

## Annotating Polygons


The GeoViews Polygons annotator behaves much like the annotator in HoloViews but also projects the coordinates in the table to be more readable:

In [ ]:
poly = gv.Polygons([sample_poly], crs=ccrs.GOOGLE_MERCATOR)

poly_annotate = annotate.instance()

annotated = poly_annotate(poly, annotations=['Value'], vertex_annotations=['Height'])

annotate.compose(tiles, annotated)

Accessing Polygons works the same as with Paths:

In [ ]:
poly_annotate.annotated.iloc[0].dframe()